In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import time
from transformers import BertModel, BertTokenizer
import torch
from torch.optim import AdamW
from transformers import BertForSequenceClassification
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score
from torch.nn import CrossEntropyLoss

In [ ]:
train_df = pd.read_csv('/content/drive/My Drive/task3_data/twitter-2016train-A.tsv', sep='\t', header=None)
val_df = pd.read_csv('/content/drive/My Drive/task3_data/twitter-2016dev-A.tsv', sep='\t', header=None)
test_df = pd.read_csv('/content/drive/My Drive/task3_data/twitter-2016test-A.tsv', sep='\t', header=None)

In [ ]:
# Generating labels for the texts
def produce_label(raw_data, one_hot=False):
    label = []
    for item in list(raw_data[1]):
        if item == 'negative':
            label.append(0)
        elif item == 'neutral':
            label.append(1)
        elif item == 'positive':
            label.append(2)
    label = np.array(label)
    if one_hot:
        oh_label = np.zeros((len(label), 3))
        for (i, item) in enumerate(label):
            oh_label[i, item] = 1.
        label = oh_label
    return label

In [ ]:
# Tokenize input and generate input ids
def produce_input(text_data, max_length=64):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    encoded_batch = tokenizer(text_data, return_tensors='pt', padding='max_length', truncation=True, max_length=max_length)
    return encoded_batch['input_ids']

In [ ]:
# Produce batch data for computing
def produce_batch_data(raw_data, batch_size=16, max_length=64, one_hot=False):
    num_length = len(raw_data)
    for n in range(0, num_length, batch_size):
        batch_data = raw_data.iloc[n:min(n+batch_size, num_length)]
        batch_label = produce_label(batch_data, one_hot)
        batch_input = [text for text in batch_data[2]]  # Assuming column 2 contains the text data
        yield batch_input, batch_label

In [ ]:
# Confirms that the batch data can be sucessfully generated
for item in produce_batch_data(train_df):
    print(item)
    break

(["dear @Microsoft the newOoffice for Mac is great and all, but no Lync update? C'mon.", "@Microsoft how about you make a system that doesn't eat my friggin discs. This is the 2nd time this has happened and I am so sick of it!", "I may be ignorant on this issue but... should we celebrate @Microsoft's parental leave changes? Doesn't the gender divide suggest... (1/2)", 'Thanks to @microsoft, I just may be switching over to @apple.', 'If I make a game as a #windows10 Universal App. Will #xboxone owners be able to download and play it in November? @majornelson @Microsoft', 'Microsoft, I may not prefer your gaming branch of business. But, you do make a damn fine operating system. #Windows10 @Microsoft', '@MikeWolf1980 @Microsoft I will be downgrading and let #Windows10 be out for almost the 1st yr b4 trying it again. #Windows10fail', '@Microsoft 2nd computer with same error!!! #Windows10fail Guess we will shelve this until SP1! http://t.co/QCcHlKuy8Q', 'Just ordered my 1st ever tablet; @Mi

In [ ]:
# Check cuda availablity
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")

Using GPU: Tesla T4


In [ ]:
# Function to convert the regression prediction back to a categorical prediction
def convert_to_categorical(predictions, threshold=0.5):
    # Assuming: 0 for 'negative', 1 for 'neutral', 2 for 'positive'
    categories = []
    for pred in predictions:
        if pred < 1 - threshold:
            categories.append(0)  # Negative
        elif pred > 1 + threshold:
            categories.append(2)  # Positive
        else:
            categories.append(1)  # Neutral
    return categories

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from torch.nn import MSELoss
from transformers import BertForSequenceClassification, AdamW
from tqdm.auto import tqdm
import torch
import numpy as np

def initialize_model():
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)
    return model.to(device)

def train_and_validate(model, train_df, val_df, device, num_epochs=10, batch_size=16):
    optimizer = AdamW(model.parameters(), lr=2e-5)
    loss_fn = MSELoss()  # Using MSE Loss

    for epoch in tqdm(range(num_epochs), desc='Epochs'):
        model.train()
        total_loss = 0
        train_predictions, train_labels = [], []
        batch_iterator = produce_batch_data(train_df, batch_size=batch_size, max_length=64, one_hot=False)

        for batch_inputs, batch_labels in tqdm(batch_iterator, desc='Batches', leave=True):
            batch_input = produce_input(batch_inputs, max_length=64)
            batch_labels = torch.tensor(batch_labels, dtype=torch.float)

            batch_input = batch_input.to(device)
            batch_labels = batch_labels.to(device)

            model.zero_grad()

            outputs = model(batch_input)
            loss = loss_fn(outputs.logits.squeeze(), batch_labels)
            total_loss += loss.item()

            categorical_preds = convert_to_categorical(outputs.logits.squeeze().detach().cpu().numpy())
            train_predictions.extend(categorical_preds)
            train_labels.extend(batch_labels.cpu().numpy())
            loss.backward()
            optimizer.step()

        avg_loss = total_loss / len(train_df) * batch_size
        train_accuracy = accuracy_score(train_labels, train_predictions)
        train_precision, train_recall, train_f1, _ = precision_recall_fscore_support(train_labels, train_predictions, average='macro', zero_division=0)

        print(f"Training Epoch {epoch} - Loss: {avg_loss:.4f}, Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1-Score: {train_f1:.4f}")

        model.eval()
        total_val_loss = 0
        val_predictions, val_labels = [], []

        with torch.no_grad():
            val_batch_iterator = produce_batch_data(val_df, batch_size=batch_size, max_length=64, one_hot=False)
            for val_batch_inputs, val_batch_labels in tqdm(val_batch_iterator, desc='Batches', leave=True):
                val_batch_input = produce_input(val_batch_inputs, max_length=64)
                val_batch_labels = torch.tensor(val_batch_labels, dtype=torch.float)

                val_batch_input = val_batch_input.to(device)
                val_batch_labels = val_batch_labels.to(device)

                val_outputs = model(val_batch_input)
                val_loss = loss_fn(val_outputs.logits.squeeze(), val_batch_labels)
                total_val_loss += val_loss.item()

                categorical_preds = convert_to_categorical(val_outputs.logits.squeeze().detach().cpu().numpy())
                val_predictions.extend(categorical_preds)
                val_labels.extend(val_batch_labels.cpu().numpy())

            avg_val_loss = total_val_loss / len(val_df) * batch_size

            accuracy = accuracy_score(val_labels, val_predictions)
            precision, recall, f1, _ = precision_recall_fscore_support(val_labels, val_predictions, average='macro', zero_division=0)

            print(f"Validation Epoch {epoch} - Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

    return model

def test_model(model, test_data, batch_size=16, max_length=64):
    model.eval()
    total_test_loss = 0
    test_predictions, test_labels = [], []
    loss_fn = MSELoss()

    with torch.no_grad():
        test_iterator = produce_batch_data(test_data, batch_size=batch_size, max_length=max_length, one_hot=False)
        for test_batch_inputs, test_batch_labels in tqdm(test_iterator, desc='Testing', leave=False):
            test_batch_input = produce_input(test_batch_inputs, max_length=max_length)
            test_batch_labels = torch.tensor(test_batch_labels, dtype=torch.float)

            test_batch_input = test_batch_input.to(device)
            test_batch_labels = test_batch_labels.to(device)

            test_outputs = model(test_batch_input)
            test_loss = loss_fn(test_outputs.logits.squeeze(), test_batch_labels)
            total_test_loss += test_loss.item()

            categorical_preds = convert_to_categorical(test_outputs.logits.squeeze().cpu().numpy())
            test_predictions.extend(categorical_preds)
            test_labels.extend(test_batch_labels.cpu().numpy())

    avg_test_loss = total_test_loss / len(test_data) * batch_size

    test_accuracy = accuracy_score(test_labels, test_predictions)
    test_precision, test_recall, test_f1, _ = precision_recall_fscore_support(test_labels, test_predictions, average='macro', zero_division=0)

    return avg_test_loss, test_accuracy, test_precision, test_recall, test_f1





In [ ]:
# Script to Run Multiple Times with Detailed Output
num_runs = 5
test_results_all_runs = []

for run in range(num_runs):
    print(f"Run {run + 1}/{num_runs}")
    model = initialize_model()  # Initialize a new model for each run
    trained_model = train_and_validate(model, train_df, val_df, device, num_epochs=10, batch_size=16)
    avg_test_loss, test_accuracy, test_precision, test_recall, test_f1 = test_model(trained_model, test_df, batch_size=16, max_length=64)
    test_results_all_runs.append((avg_test_loss, test_accuracy, test_precision, test_recall, test_f1))

    # Print results for the current run
    print(f"Results for Run {run + 1}:")
    print(f"  Test Loss: {avg_test_loss:.4f}")
    print(f"  Test Accuracy: {test_accuracy:.4f}")
    print(f"  Test Precision: {test_precision:.4f}")
    print(f"  Test Recall: {test_recall:.4f}")
    print(f"  Test F1-Score: {test_f1:.4f}\n")

# Convert list to numpy array for easier calculation
test_results_all_runs = np.array(test_results_all_runs)

# Calculate mean and standard deviation across runs
mean_test_results = np.mean(test_results_all_runs, axis=0)
std_test_results = np.std(test_results_all_runs, axis=0)

# Print the averaged results
print("Averaged Test Results Across Runs:")
print(f"  Mean Test Loss: {mean_test_results[0]:.4f} (±{std_test_results[0]:.4f})")
print(f"  Mean Test Accuracy: {mean_test_results[1]:.4f} (±{std_test_results[1]:.4f})")
print(f"  Mean Test Precision: {mean_test_results[2]:.4f} (±{std_test_results[2]:.4f})")
print(f"  Mean Test Recall: {mean_test_results[3]:.4f} (±{std_test_results[3]:.4f})")
print(f"  Mean Test F1-Score: {mean_test_results[4]:.4f} (±{std_test_results[4]:.4f})")

Run 1/5


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

...

Batches: 0it [00:00, ?it/s]

Training Epoch 0 - Loss: 0.5489, Accuracy: 0.4451, Precision: 0.4681, Recall: 0.3793, F1-Score: 0.3438


Batches: 0it [00:00, ?it/s]

Validation Epoch 0 - Loss: 0.4207, Accuracy: 0.5488, Precision: 0.3664, Recall: 0.4540, F1-Score: 0.4052


Batches: 0it [00:00, ?it/s]